<a href="https://colab.research.google.com/github/ougrid/my-knowledge-resource/blob/master/TextPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install pythainlp
!pip install gensim==4.2.0
!pip install transformers
!pip install sentencepiece

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wisesight_sentiment")

Generating train split:   0%|          | 0/21628 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2404 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2671 [00:00<?, ? examples/s]

Dataset wisesight_sentiment downloaded and prepared to /root/.cache/huggingface/datasets/wisesight_sentiment/wisesight_sentiment/1.0.0/fc2b1bdfe79571b2e281e4afdb5aac069cf9270bf0f85694239be672a4191969. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
df=pd.DataFrame(dataset["train"])

In [ ]:
df=df.iloc[:5000]

In [ ]:
df

In [ ]:
for i in df["texts"]:
  print(i)

In [ ]:
import re

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)



In [ ]:
from pythainlp.util import normalize

In [ ]:
def clean_text(text):

  # emoji
  text = emoji_pattern.sub("",text)

  # hashtag
  text = re.sub("#[a-zA-Zก-๙0-9_]+","",text)

  # ภาษาไทย ภาษาอังกฤษ ตัวเลข .
  text = re.sub("[^a-zA-Zก-๙0-9\. ]","",text)

  # 55555+ , 5555555555 => 555
  text = re.sub("5+\+|5{3,}","555",text)

  text = normalize(text)

  return text



df["cleaned_text"]=df["texts"].apply(clean_text)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df=df[df["cleaned_text"]!=""]

In [ ]:
df

# Simple Logistic Regression

Word tokenize => Word Embedding => Logistic Regression

In [ ]:
df

In [ ]:
from pythainlp.word_vector import WordVector

wv=WordVector()


X = df["cleaned_text"].apply(wv.sentence_vectorizer)
y = df["category"]

Corpus: thai2fit_wv
- Downloading: thai2fit_wv 0.1


  0%|          | 0/62452646 [00:00<?, ?it/s]

In [ ]:
X= [x.reshape(300,) for x in X]

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X, y)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
clf.predict(wv.sentence_vectorizer("คุณพี่อยู่จังหวัดอะไรคะ"))

array([1])

# Inference Using Transformers

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModel,
    pipeline,
)

In [ ]:
model_name = "wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(
                f'airesearch/{model_name}',
                revision='main',
                model_max_length=416,)

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

In [ ]:
dataset_name = "wisesight_sentiment" 

#pipeline
classify_multiclass = pipeline(task='sentiment-analysis',
         tokenizer=tokenizer,
         model = f'airesearch/{model_name}',
         revision = f'finetuned@{dataset_name}')

Downloading:   0%|          | 0.00/716 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

In [ ]:
input_text = "ผมพูดเล่นจริงๆ แต่คุณพูดจริงเล่นๆ"

classify_multiclass(input_text)

[{'label': 'neu', 'score': 0.8019969463348389}]

# Simpletransformer

In [ ]:
!pip install simpletransformers

In [ ]:
train_df = df[["cleaned_text","category"]]
train_df

In [ ]:
!rm -rf /content/outputs

In [ ]:
# GOGO Bumblebee

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data

train_df = df[["cleaned_text","category"]]
train_df.columns = ["text", "labels"]

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=5000)

# Create a ClassificationModel
model = ClassificationModel(
    'camembert',
    'airesearch/wangchanberta-base-att-spm-uncased',
    num_labels=4,
    args=model_args
) 

# Train the model
model.train_model(train_df)





In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["ไอ้กระจอก"])

In [ ]:
predictions

In [ ]:
# https://attapol.github.io/compling/index.html
# youtube ekapolcn https://www.youtube.com/c/EkapolC/playlists
# fbpage https://www.facebook.com/groups/thainlp